In [3]:
import pandas as pd
df = pd.read_csv("./데이터/vip_customers_all.csv")

/var/folders/3p/8hc9c44s38j902ytq253kt6m0000gn/T/ipykernel_2556/3880923711.py:2: DtypeWarning: Columns (339) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./데이터/vip_customers_all.csv")


In [4]:
df.head()

,기준년월,발급회원번호,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,SYN_0,0.454549,0.454549,0.364728,2.0,0.0,0.000000,0.0,0.409767,...,0,0,0,0,0,0,0,0,5.자녀성장기(2),22
1,201807,SYN_1000,0.204653,0.204653,0.204653,0.0,0.0,0.000000,0.0,1.411954,...,0,0,0,0,0,0,0,0,4.자녀성장기(1),45
2,201807,SYN_1005,0.148929,0.148929,0.099295,2.0,0.0,-0.019939,0.0,0.357926,...,0,0,0,0,0,0,0,1,2.가족형성기,8
3,201807,SYN_1007,-0.189322,-0.189322,-0.189322,0.0,0.0,0.000000,0.0,0.189824,...,0,0,0,0,0,0,0,0,3.자녀출산기,0
4,201807,SYN_1009,0.040080,0.040080,0.040080,0.0,0.0,-0.243847,0.0,0.204423,...,0,0,0,0,0,0,0,0,5.자녀성장기(2),22


In [5]:
len(df)

11460

In [2]:
df = df.sort_values(by=['발급회원번호', '기준년월'])

In [3]:
def calculate_slope(series):
    # 데이터가 너무 적으면(2달 미만) 추세를 못 구하니까 0 처리
    if len(series) < 2: return 0
    
    # x축은 시간(0, 1, 2...), y축은 소비금액
    # linregress가 데이터 점들을 가장 잘 설명하는 '직선'을 긋고, 그 '기울기'를 알려줘.
    slope, _, _, _, _ = linregress(np.arange(len(series)), series.values)
    return slope

In [4]:
# 고객(발급회원번호)별로 묶어서 하나씩 계산
for customer_id, group_data in df.groupby('발급회원번호'):
    
    # 1. 이용금액: 돈을 점점 안 쓰나?
    slope_spend = calculate_slope(group_data['이용금액_신용_B0M'])
    
    # 2. 이용건수: 카드 긁는 횟수가 줄어드나?
    slope_count = calculate_slope(group_data['이용건수_신용_B0M'])
    
    # 3. 잔액: 남아있는 카드값(잔액)도 줄어드나? (아예 안 쓴다는 신호)
    slope_balance = calculate_slope(group_data['잔액_B0M'])
    
    # 이 3가지 결과를 저장!

In [ ]:
# 정규화(Scaling)를 위해 변수 이름들을 리스트로 정의
cols = ['Slope_Spend', 'Slope_Count', 'Slope_Balance']

# 정규화: 모든 기울기를 0~1 사이 숫자로 통일
slope_scaled = scaler.fit_transform(slope_df[cols])

# 점수 계산 공식: (1 - 기울기) * 가중치
# 기울기가 뚝뚝 떨어질수록(값이 작을수록) -> 점수는 높게(100점에 가깝게) 나옴!
# 가중치: 금액(40%) > 건수(30%) = 잔액(30%)
slope_df['Churn_Score'] = (
    (1 - slope_scaled[:, 0]) * 0.4 + 
    (1 - slope_scaled[:, 1]) * 0.3 + 
    (1 - slope_scaled[:, 2]) * 0.3
) * 100

In [9]:
# ---------------------------------------------------------
# 5. [핵심] 클러스터링으로 이탈 기준선(Threshold) 찾기
# ---------------------------------------------------------
print("🤖 AI(K-Means)가 점수 분포를 보고 이탈 기준선을 찾습니다...")

from sklearn.cluster import KMeans

# (1) 데이터 준비: AI한테 던져줄 때는 2차원 배열(표 형태)이어야 해서 .values로 뽑아줍니다.
X = slope_df[['Churn_Score']].values

# (2) 모델 설정: 
# "데이터를 딱 2개 그룹(안전한 그룹 vs 위험한 그룹)으로 나눠줘!" (n_clusters=2)
# "매번 똑같은 결과가 나오게 고정해줘!" (random_state=42)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)

# (3) 학습(Fit): 실제로 데이터를 씹어먹고 그룹을 나눕니다.
kmeans.fit(X)

# (4) 결과 확인: AI가 나눈 두 그룹의 '중심점(평균 점수)'을 확인합니다.
centers = sorted(kmeans.cluster_centers_.flatten())
safe_center = centers[0]  # 점수가 낮은 그룹의 중심 (안전)
risk_center = centers[1]  # 점수가 높은 그룹의 중심 (위험)

# (5) 기준선(Threshold) 확정:
# 두 그룹의 딱 '중간 지점'을 이탈과 유지를 가르는 경계선으로 잡습니다.
threshold_score = (safe_center + risk_center) / 2

print(f"   - 안전 그룹 평균: {safe_center:.2f}점")
print(f"   - 위험 그룹 평균: {risk_center:.2f}점")
print(f"👉 AI가 정해준 이탈 기준 점수: {threshold_score:.2f}점")

🤖 AI(K-Means)가 점수 분포를 보고 이탈 기준선을 찾습니다...
   - 안전 그룹 평균: 55.78점
   - 위험 그룹 평균: 65.37점
👉 AI가 정해준 이탈 기준 점수: 60.57점


In [10]:
# ---------------------------------------------------------
# 6. 최종 타깃(Target) 확정 (0 또는 1)
# ---------------------------------------------------------
print("🎯 기준 점수를 넘는 고객을 '이탈(1)'로 확정합니다!")

# [핵심 로직]
# 조건: 이탈 위험 점수(Churn_Score)가 기준선(threshold_score)보다 크거나 같으면?
# 결과: True면 1(이탈), False면 0(유지)으로 변환 (.astype(int))
slope_df['Target'] = (slope_df['Churn_Score'] >= threshold_score).astype(int)

# 통계 확인: 몇 명이나 잡혔는지 봅니다.
churn_count = slope_df['Target'].sum()   # 이탈자 수
total_count = len(slope_df)              # 전체 인원
churn_rate = churn_count / total_count * 100  # 이탈률

print("\n" + "="*40)
print(f"📊 [최종 결과 리포트]")
print(f"   - 분석 대상: {total_count:,}명")
print(f"   - 이탈 확정(1): {churn_count:,}명 ({churn_rate:.2f}%)")
print(f"   - 유지(0): {total_count - churn_count:,}명 ({100 - churn_rate:.2f}%)")
print("="*40)

🎯 기준 점수를 넘는 고객을 '이탈(1)'로 확정합니다!

📊 [최종 결과 리포트]
   - 분석 대상: 1,910명
   - 이탈 확정(1): 854명 (44.71%)
   - 유지(0): 1,056명 (55.29%)
